In [1]:
from __future__ import division

import csv
import numpy as np
import random
from random import randrange
from math import exp
import copy

In [2]:
data = []
with open('irisData.csv', 'rb') as csv_file:
    iris_data_reader = csv.reader(csv_file, delimiter=',', quotechar='|')
    for row in iris_data_reader:
        if row[4] == 'Iris-setosa':
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 1])
        elif row[4] == 'Iris-versicolor':
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 2])
        else:
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 3])

In [3]:
def normalize_dataset(data):
    minmax = list()
    for i in range(len(data[0])):
        col_values = [row[i] for row in data]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])

    for row in data:
        for i in range(len(row) - 1):
            if minmax[i][1] != minmax[i][0]:
                row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


normalize_dataset(data)
np.random.shuffle(data)
training_data = data[:135]
test_data = data[135:150]

In [4]:
def logistic_function(x):
    return 1.0 / (1.0 + exp((-1) * x))
    
def coefficients_sgd(train, learning_rate, n_epoch, learning_class, regularization_rate):
    """Calculates coefficients for vector (which will be used in logistic regression function)
    Use stochastic gradient algorithm: select random vector from training data, calculate gradient on this vector
    and change separating vector.

    train -- train data
    learning_rate --  Used to limit the amount each coefficient is corrected each time it is updated.
    n_epoch -- count of learning epochs
    learning_class -- class which will be learned (there is tree classes for iris data)
    regularization_rate -- parameter which will be used for l2 regularization
    """
    w = np.array([0, 0, 0, 0, 0], dtype=np.float32)
    for epoch in range(n_epoch):
        random_row = random.choice(train)
        temp_w = copy.copy(w)
        if learning_class == random_row[5]:
            y = 1
        else:
            y = -1
        for i in range(0, 5):
            w[i] = w[i] - learning_rate * ((1 - logistic_function(np.sum(random_row[0:5] * temp_w) * y))
                                           * ((-1) * y * random_row[i]) + regularization_rate * w[i])
    return w


def coefficients_batchgd(train, learning_rate, n_epoch, learning_class, regularization_rate):
    """Calculates coefficients for vector (which will be used in logistic regression function)
       Use batch gradient algorithm: calculates average gradient for all samples and change vector.

       train -- train data
       learning_rate --  Used to limit the amount each coefficient is corrected each time it is updated.
       n_epoch -- count of learning epochs
       learning_class -- class which will be learned (there is tree classes for iris data)
       regularization_rate -- parameter which will be used for l2 regularization
       """
    w = np.array([0, 0, 0, 0, 0], dtype=np.float32)
    for epoch in range(n_epoch):
        w_accumulate = np.array([0, 0, 0, 0, 0], dtype=np.float32)
        temp_w = copy.copy(w)
        for row in train:
            if learning_class == row[5]:
                y = 1
            else:
                y = -1
            for i in range(0, 5):
                w_accumulate[i] += ((1 - logistic_function(np.sum(row[0:5] * temp_w) * y))
                                    * ((-1) * y * row[i]) + regularization_rate * w[i])
        for i in range(0, 5):
            w[i] = w[i] - learning_rate * w_accumulate[i] / train.__len__()
    return w

In [5]:
def cross_validation_split(dataset, n_folds):
    """ Split data set for n folds. Need for k-fold validation"""
    dataset_split = []
    dataset_copy = copy.copy(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


def test_prediction(sample, w, learning_class):
    """Predict result on sample and returns result according learning class"""
    prediction = logistic_function(np.sum(sample[0:5] * w))
    if prediction >= 0.5 and learning_class == sample[5] or prediction < 0.5 and learning_class != sample[5]:
        return 1
    else:
        return 0


def test_solution(test_data, w, learning_class):
    """Calculates percentage of correctly predicted objects"""
    correctly_predicted = 0
    for test_row in test_data:
        correctly_predicted += test_prediction(test_row, w, learning_class)
    return correctly_predicted / test_data.__len__()


In [7]:
def evaluate_algorithm(algorithm, dataset, n_epochs, learning_rate, learning_class, regularization_rate, n_folds=5):
    """ Evaluate one of two algorithms (stochastic or batch gradient descent)
    
    algorithm -- coefficients_sgd or coefficients_batchgd
    dataset -- training data
    n_epochs -- number epochs for train
    learning_rate --  Used to limit the amount each coefficient is corrected each time it is updated
    learning_class -- class which will be learned (there is tree classes for iris data)
    regularization_rate -- parameter which will be used for l2 regularization
    n_folds -- number of folds for k-fold algorithm   
    """
    
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for test_set in folds:
        train_set = [item for item in dataset if item not in test_set]
        w = algorithm(train_set, learning_rate, n_epochs, learning_class, regularization_rate)
        accuracy = test_solution(test_set, w, learning_class)
        scores.append(accuracy)
    return sum(scores)/scores.__len__()


In [50]:
# Find solution with stochastic gradient descent
best_solution = 0
best_params = None
# for n_epochs in range(100, 200, 500):
for n_epochs in [250]:
    for learning_rate in np.arange(0.232, 0.233, 0.00001):
        for regularization_rate in np.arange(0.0, 0.001, 0.0001):
            class1_accuracy = evaluate_algorithm(coefficients_sgd, training_data, n_epochs, learning_rate, 1,
                                                 regularization_rate)
            class2_accuracy = evaluate_algorithm(coefficients_sgd, training_data, n_epochs, learning_rate, 2,
                                                 regularization_rate)
            class3_accuracy = evaluate_algorithm(coefficients_sgd, training_data, n_epochs, learning_rate, 3,
                                                 regularization_rate)
            print class1_accuracy
            print class2_accuracy
            print class3_accuracy

            if class1_accuracy + class2_accuracy + class3_accuracy > best_solution:
                best_solution = class1_accuracy + class2_accuracy + class3_accuracy
                best_params = (n_epochs, learning_rate, regularization_rate)
            print regularization_rate
print best_params
print best_solution


1.0
0.637037037037
0.851851851852
0.0
1.0
0.681481481481
0.814814814815
0.0001


1.0
0.651851851852
0.925925925926
0.0002
1.0
0.637037037037
0.903703703704
0.0003


1.0
0.644444444444
0.874074074074
0.0004
1.0
0.637037037037
0.762962962963
0.0005


1.0
0.681481481481
0.822222222222
0.0006
1.0
0.644444444444
0.911111111111
0.0007


1.0
0.622222222222
0.925925925926
0.0008
1.0
0.666666666667
0.807407407407
0.0009


...

1.0
0.622222222222
0.866666666667
0.0
1.0
0.659259259259
0.807407407407
0.0001


1.0
0.659259259259
0.911111111111
0.0002
1.0
0.644444444444
0.881481481481
0.0003


1.0
0.681481481481
0.911111111111
0.0004
1.0
0.6
0.851851851852
0.0005


1.0
0.703703703704
0.866666666667
0.0006
1.0
0.62962962963
0.925925925926
0.0007


1.0
0.644444444444
0.896296296296
0.0008
1.0
0.681481481481
0.874074074074
0.0009
(250, 0.2320900000000001, 0.00050000000000000001)
2.64444444444


Запускал k-fold с разными значениями параметров для количества итерция, параметра градиентного спуска и параметра регуляризация.
Лучшие результаты алгоритма получались при болом значении парметра кол-ва эпох, поэтому проводил далее эксперементы только для большлго значения этого параметра.
значение параметра uрадиентного спуска было  лучшим в пределах от 0 до 0.5. Поэтому ,постепенно уменьшая интервал, пришёл к значению это параметра равному 0.232.
Регуляризация улучшала решение только при маленьких значениях, поэтому также постпенно уменьшая интревал (в k-fold) выбрал 0.0006 в качестве этого параметра.

Но для сторого класса ирисов (Iris-versicolor) алгоритм работает плохо у меня..

In [61]:
class1_accuracy = evaluate_algorithm(coefficients_sgd, training_data, 400, 0.2324, 1, 0.0006)
class2_accuracy = evaluate_algorithm(coefficients_sgd, training_data, 400, 0.2324, 2, 0.0006)
class3_accuracy = evaluate_algorithm(coefficients_sgd, training_data, 400, 0.2324, 3, 0.0006)
        
print "Результат при выбранных параметрах алгоритма на тренировочных данных"
print class1_accuracy
print class2_accuracy
print class3_accuracy

        
print "Результат при выбранных параметрах алгоритма на тестовых данных"
class1_accuracy = evaluate_algorithm(coefficients_sgd, test_data, 400, 0.2324, 1, 0.0006)
class2_accuracy = evaluate_algorithm(coefficients_sgd, test_data, 400, 0.2324, 2, 0.0006)
class3_accuracy = evaluate_algorithm(coefficients_sgd, test_data, 400, 0.2324, 3, 0.0006)
        
print class1_accuracy
print class2_accuracy
print class3_accuracy

Результат при выбранных параметрах алгоритма на тренировочных данных
1.0
0.592592592593
0.925925925926
Результат при выбранных параметрах алгоритма на тестовых данных


1.0
0.733333333333
0.933333333333


In [70]:
# Find solution with batch gradient descent
best_solution = 0
best_params = None
# for n_epochs in range(100, 200, 500):
for n_epochs in [100]:
    for learning_rate in np.arange(0.22, 0.23, 0.001):
        for regularization_rate in np.arange(0.0, 0.001, 0.0001):
            class1_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, n_epochs, learning_rate, 1,
                                                 regularization_rate)
            class2_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, n_epochs, learning_rate, 2,
                                                 regularization_rate)
            class3_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, n_epochs, learning_rate, 3,
                                                 regularization_rate)
            print class1_accuracy
            print class2_accuracy
            print class3_accuracy
    
            if class1_accuracy + class2_accuracy + class3_accuracy > best_solution:
                best_solution = class1_accuracy + class2_accuracy + class3_accuracy
                best_params = (n_epochs, learning_rate, regularization_rate)
            print regularization_rate
print best_params
print best_solution

1.0
0.681481481481
0.955555555556
0.0009


1.0
0.681481481481
0.955555555556
0.0009


1.0
0.681481481481
0.866666666667
0.0009


1.0
0.681481481481
0.940740740741
0.0009


1.0
0.681481481481
0.940740740741
0.0009


...

1.0
0.681481481481
0.955555555556
0.0009


1.0
0.681481481481
0.918518518519
0.0009


1.0
0.681481481481
0.933333333333
0.0009


1.0
0.681481481481
0.918518518519
0.0009


1.0
0.681481481481
0.933333333333
0.0009
(100, 0.22, 0.00090000000000000008)
2.63703703704


Выбирал параметры по такому же принципу как и в тохастическом градиентном спуске.
Только кол-во эпо брал меньше, так как очевидно, что тут вычислений больше.
Кол-во эпох выбрал = 100
Параметр градиента 0.22
Регуляризация 0.001

In [ ]:
class1_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, 100, 0.22, 1, 0.0009)
class2_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, 100, 0.22, 2, 0.0009)
class3_accuracy = evaluate_algorithm(coefficients_batchgd, training_data, 100, 0.22, 3, 0.0009)
        
print "Результат при выбранных параметрах алгоритма на тренировочных данных"
print class1_accuracy
print class2_accuracy
print class3_accuracy

        
print "Результат при выбранных параметрах алгоритма на тестовых данных"
class1_accuracy = evaluate_algorithm(coefficients_batchgd, test_data, 100, 0.22, 1, 0.0009)
class2_accuracy = evaluate_algorithm(coefficients_batchgd, test_data, 100, 0.22, 2, 0.0009)
class3_accuracy = evaluate_algorithm(coefficients_batchgd, test_data, 100, 0.22, 3, 0.0009)
        
print class1_accuracy
print class2_accuracy
print class3_accuracy

Результат при выбранных параметрах алгоритма на тренировочных данных
1.0
0.681481481481
0.925925925926
Результат при выбранных параметрах алгоритма на тестовых данных


0.933333333333
0.4
0.733333333333


Стохастический алгоритм считает градиент на одном случаном элементе, он работает лучше при малых объёмах выборки, также он работает быстрее

Batch gradient descent считает градиент на всех обучающейся выборке, поэтому он как бы движется к оптимальному решению. При больших объёмах выборкипользоваться этималгоримом затруднительно, так как это потребует много времени